In [1]:
import sys
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import LabelEncoder

# RecSys Challenge 2015のデータセット

In [2]:
train_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-clicks.dat", header=None)
test_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-test.dat", header=None)

/tmp/ipykernel_291/2692782158.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-clicks.dat", header=None)
/tmp/ipykernel_291/2692782158.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-test.dat", header=None)


In [3]:
train_df.columns = ["sessionId", "timestamp", "itemId", "categoryId"]
test_df.columns = ["sessionId", "timestamp", "itemId", "categoryId"]

In [4]:
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%fZ")
test_df["timestamp"] = pd.to_datetime(test_df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%fZ")

In [5]:
# アイテム数が4より小さいものを削除
remove_items = train_df.groupby("itemId").size()
remove_items = remove_items[remove_items > 4].index
train_df = train_df[train_df["itemId"].isin(remove_items)]

In [6]:
# sessionの長さが1のものを消去
train_session_len = train_df.groupby("sessionId").size()
train_df = train_df[train_df["sessionId"].isin(train_session_len[train_session_len > 1].index)]

In [7]:
train_df = train_df.sort_values("timestamp")
train_df

,sessionId,timestamp,itemId,categoryId
1031562,351646,2014-04-01 03:00:00.124,214717005,0
1163646,389654,2014-04-01 03:00:00.567,214826705,0
885068,263073,2014-04-01 03:00:10.087,214716982,0
620662,210798,2014-04-01 03:00:13.070,214581827,0
1217521,375257,2014-04-01 03:00:13.768,214644307,0
...,...,...,...,...
32350125,11528554,2014-09-30 02:59:43.577,214572433,S
32653390,11422848,2014-09-30 02:59:47.397,214859908,S
32432376,11474968,2014-09-30 02:59:49.546,214685049,3
32432377,11474968,2014-09-30 02:59:53.315,214685049,3


In [8]:
# テストデータからトレインデータに含まれないアイテムを除外
test_df = test_df[test_df["itemId"].isin(train_df["itemId"].unique())]

In [9]:
# テストデータからセッション数が1のものを削除
test_session_len = test_df.groupby("sessionId").size()
test_df = test_df[test_df.sessionId.isin(test_session_len[test_session_len > 1].index)]

In [10]:
df = pd.concat([train_df, test_df], axis=0)

In [11]:
df

,sessionId,timestamp,itemId,categoryId
1031562,351646,2014-04-01 03:00:00.124,214717005,0
1163646,389654,2014-04-01 03:00:00.567,214826705,0
885068,263073,2014-04-01 03:00:10.087,214716982,0
620662,210798,2014-04-01 03:00:13.070,214581827,0
1217521,375257,2014-04-01 03:00:13.768,214644307,0
...,...,...,...,...
8251786,11299820,2014-09-25 08:17:19.053,214853094,3
8251787,11299815,2014-09-26 09:49:23.602,214854804,S
8251788,11299815,2014-09-26 09:49:41.808,214714715,S
8251789,11299810,2014-09-26 16:00:32.051,214546123,2


In [12]:
item_encoder = LabelEncoder()
df["itemId"] = item_encoder.fit_transform(df["itemId"])
train_df["itemId"] = item_encoder.transform(train_df["itemId"])
test_df["itemId"] = item_encoder.transform(test_df["itemId"])

In [13]:
session_encoder = LabelEncoder()
df["sessionId"] = session_encoder.fit_transform(df["sessionId"])
train_df["sessionId"] = session_encoder.transform(train_df["sessionId"])
test_df["sessionId"] = session_encoder.transform(test_df["sessionId"])

In [14]:
train_df.to_pickle("/home/inoue/work/dataset/RC15/derived/train.df")
test_df.to_pickle("/home/inoue/work/dataset/RC15/derived/test.df")